In [1]:
import sys
import numpy as np
import torch
import os, glob
import smplx

In [2]:
from GRAB.tools.objectmodel import ObjectModel
from GRAB.tools.utils import parse_npz, prepare_params, params2torch, to_cpu
from GRAB.tools.meshviewer import Mesh


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = os.path.abspath('./data/grab_unzipped/grab')
out_dir = os.path.abspath('./data/grab_preprocessed')
model_path = os.path.abspath('./body_models/')

os.makedirs(out_dir, exist_ok=True)

print('Using device:', device)
print('Data directory:', data_dir)
print('Output directory:', out_dir)
print('SMPLX/MANO Model directory:', model_path)

Using device: cuda
Data directory: /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab
Output directory: /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_preprocessed
SMPLX/MANO Model directory: /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/body_models


In [4]:
# map directory names to lists of files in them
data_dict = {}

for root, dirs, files in os.walk(data_dir):
    if len(files) > 0:
        print(f'Adding directory {root} with {len(files)} files, creating output folder structure...')
        data_dict[root] = []

        for file in files:
            file_in = os.path.join(root, file)
            file_out_dir = os.path.join(out_dir, f'{os.path.basename(root)}_{os.path.splitext(file)[0]}')
            os.makedirs(file_out_dir, exist_ok=True)
            data_dict[root].append((file_in, file_out_dir))



Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s5 with 106 files, creating output folder structure...
Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s4 with 113 files, creating output folder structure...
Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s1 with 198 files, creating output folder structure...
Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s9 with 125 files, creating output folder structure...
Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s3 with 125 files, creating output folder structure...
Adding directory /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s7 with 120 files

In [10]:
def load_sbj_verts(seq_data):
        mesh_path = os.path.join(data_dir, '..',seq_data.body.vtemp)
        sbj_vtemp = np.array(Mesh(filename=mesh_path).vertices)
        return sbj_vtemp

def load_obj_verts(obj_name, seq_data, n_verts_sample=512):
    mesh_path = os.path.join(data_dir, '..',seq_data.object.object_mesh)
    np.random.seed(100)
    obj_mesh = Mesh(filename=mesh_path)
    verts_obj = np.array(obj_mesh.vertices)
    faces_obj = np.array(obj_mesh.faces)

    if verts_obj.shape[0] > n_verts_sample:
        verts_sample_id = np.random.choice(verts_obj.shape[0], n_verts_sample, replace=False)
    else:
        verts_sample_id = np.arange(verts_obj.shape[0])

    verts_sampled = verts_obj[verts_sample_id]
    obj_info = {'verts': verts_obj,
                'faces': faces_obj,
                'verts_sample_id': verts_sample_id,
                'verts_sample': verts_sampled,
                'obj_mesh_file': mesh_path}

    return obj_info

save_body_verts = True
save_lhand_verts = True
save_rhand_verts = True
save_object_verts = True
save_contact = True
n_verts_sample = 1000


def process_data_entry(root, in_file, out_dir):
    body_data = {
        'global_orient': [],'body_pose': [],'transl': [],
        'right_hand_pose': [],'left_hand_pose': [],
        'jaw_pose': [],'leye_pose': [],'reye_pose': [],
        'expression': [],'fullpose': [],
        'contact':[], 'verts' :[]
    }

    object_data = {'verts': [], 'global_orient': [], 'transl': [], 'contact': []}
    lhand_data = {'verts': [], 'global_orient': [], 'hand_pose': [], 'transl': [], 'fullpose': []}
    rhand_data = {'verts': [], 'global_orient': [], 'hand_pose': [], 'transl': [], 'fullpose': []}

    seq_data = parse_npz(in_file)

    obj_name = seq_data.obj_name
    sbj_id   = seq_data.sbj_id
    n_comps  = seq_data.n_comps
    gender   = seq_data.gender

    # need this for other methods from GRAB, this should not filter out any frames
    frame_mask = (seq_data['contact']['object']>-1).any(axis=1)
    T = frame_mask.sum()

    sbj_params = prepare_params(seq_data.body.params, frame_mask)
    rh_params  = prepare_params(seq_data.rhand.params, frame_mask)
    lh_params  = prepare_params(seq_data.lhand.params, frame_mask)
    obj_params = prepare_params(seq_data.object.params, frame_mask)

    sbj_vtemp = load_sbj_verts(seq_data)

    if save_body_verts:

        sbj_m = smplx.create(model_path=model_path,
                                model_type='smplx',
                                gender=gender,
                                num_pca_comps=n_comps,
                                v_template=sbj_vtemp,
                                batch_size=T)

        sbj_parms = params2torch(sbj_params)
        verts_sbj = to_cpu(sbj_m(**sbj_parms).vertices)
        body_data['verts'].append(verts_sbj)

    if save_lhand_verts:
        lh_mesh = os.path.join(data_dir, '..', seq_data.lhand.vtemp)
        lh_vtemp = np.array(Mesh(filename=lh_mesh).vertices)

        lh_m = smplx.create(model_path=model_path,
                            model_type='mano',
                            is_rhand=False,
                            v_template=lh_vtemp,
                            num_pca_comps=n_comps,
                            flat_hand_mean=True,
                            batch_size=T)

        lh_parms = params2torch(lh_params)
        verts_lh = to_cpu(lh_m(**lh_parms).vertices)
        lhand_data['verts'].append(verts_lh)

    if save_rhand_verts:
        rh_mesh = os.path.join(data_dir, '..', seq_data.rhand.vtemp)
        rh_vtemp = np.array(Mesh(filename=rh_mesh).vertices)

        rh_m = smplx.create(model_path=model_path,
                            model_type='mano',
                            is_rhand=True,
                            v_template=rh_vtemp,
                            num_pca_comps=n_comps,
                            flat_hand_mean=True,
                            batch_size=T)

        rh_parms = params2torch(rh_params)
        verts_rh = to_cpu(rh_m(**rh_parms).vertices)
        rhand_data['verts'].append(verts_rh)

    ### for objects

    obj_info = load_obj_verts(obj_name, seq_data, n_verts_sample)

    if save_object_verts:

        obj_m = ObjectModel(v_template=obj_info['verts_sample'],
                            batch_size=T)
        obj_parms = params2torch(obj_params)
        verts_obj = to_cpu(obj_m(**obj_parms).vertices)
        object_data['verts'].append(verts_obj)

    if save_contact:
        body_data['contact'].append(seq_data.contact.body[frame_mask])
        object_data['contact'].append(seq_data.contact.object[frame_mask][:,obj_info['verts_sample_id']])

    out_data = [body_data, lhand_data, rhand_data, object_data]
    out_data_name = ['body_data', 'rhand_data', 'lhand_data', 'object_data']
    # save with numpy npz
    for i in range(len(out_data)):
        out_file = os.path.join(out_dir, f'{out_data_name[i]}.npz')
        np.savez_compressed(out_file, **out_data[i])

    print(f'Processed {in_file} and saved to {out_dir}')


In [11]:
from tqdm import tqdm
# iterate over all files and process them, show tqdm progress bar
for root, files in data_dict.items():
    for in_file, out_dir in tqdm(files):
        process_data_entry(root, in_file, out_dir)

  0%|          | 0/106 [00:00<?, ?it/s]

  1%|          | 1/106 [00:01<02:24,  1.38s/it]

Processed /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s5/flashlight_on_2.npz and saved to /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_preprocessed/s5_flashlight_on_2


  2%|▏         | 2/106 [00:04<03:42,  2.14s/it]

Processed /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s5/cylindermedium_lift.npz and saved to /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_preprocessed/s5_cylindermedium_lift


...

Processed /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s4/camera_pass_1.npz and saved to /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_preprocessed/s4_camera_pass_1


 33%|███▎      | 37/113 [00:54<01:41,  1.34s/it]

Processed /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_unzipped/grab/s4/cubelarge_pass_1.npz and saved to /home/erik/ethz/digital-humans/dex-hoi/data_preprocessing/grab_preprocessing/data/grab_preprocessed/s4_cubelarge_pass_1


 33%|███▎      | 37/113 [00:55<01:54,  1.51s/it]

OSError: [Errno 28] No space left on device